In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords

In [2]:
Data = pd.read_csv('TrainData.csv', encoding = "ISO-8859-1" )

In [3]:
Data.Sentiment.replace({"Negative ":"Negative"},inplace=True)

In [4]:
Data['Sentiment'] = Data['Sentiment'].fillna(Data['Sentiment'].mode()[0])

In [5]:

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\45448\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\45448\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\45448\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [6]:
import textblob
from sklearn.metrics import confusion_matrix, classification_report
np.set_printoptions(precision=2, linewidth=80)

In [7]:
Data

,Review,Sentiment
0,Update : Now that the tech issues have been ad...,Positive
1,Changing my rating since the problems are fixe...,Positive
2,App worked pretty much as expected. Could have...,Positive
3,Highly recommend when you travel with Air Cana...,Positive
4,The app went from very useful to absolutely us...,Positive
...,...,...
346,This app works OK generally but I have been ha...,Negative
347,Whenever I get a notification about a flight f...,Negative
348,I travel a lot on a number of airlines. Delta ...,Positive
349,Had to keep adding my flight info so I could u...,Negative


In [9]:
reviews = np.array(Data['Review'])
sentiments = np.array(Data['Sentiment'])

# extract data for model evaluation
test_reviews = reviews[249:]
test_sentiments = sentiments[249:]


In [12]:
sample_review_ids = [56, 86, 100]

In [30]:
for review, sentiment in zip(test_reviews[sample_review_ids], test_sentiments[sample_review_ids]):
    print('REVIEW:', review)
    print('Actual Sentiment:', sentiment)
    print('Predicted Sentiment polarity:', textblob.TextBlob(review).sentiment.polarity)
    print('-'*60)

REVIEW: This app sucks. My flight was canceled and decided to download the app to keep current on my connecting flight status. I ultimately missed my connection and was automatically rebooked on the next available flight twice. I only knew this because I was looking directly online. the app still had me on the first flight I missed and if I relied on the app, I would have had no clue what was going on. not to mention it is hard to navigate. I am deleting the app and going to rely on the website.
Actual Sentiment: Negative
Predicted Sentiment polarity: 0.017592592592592597
------------------------------------------------------------
REVIEW: As others have stated, app loses flight info shortly before takeoff. Crashes when attempting to view boarding pass. Frustrating...
Actual Sentiment: Negative
Predicted Sentiment polarity: -0.2333333333333333
------------------------------------------------------------
REVIEW: Had to keep adding my flight info so I could use boarding passes at each le

In [31]:
sentiment_polarity = [textblob.TextBlob(review).sentiment.polarity for review in test_reviews]

In [32]:
predicted_sentiments = ['positive' if score >= 0.1 else 'negative' for score in sentiment_polarity]

In [29]:
for review, sentiment in zip(test_reviews[sample_review_ids], test_sentiments[sample_review_ids]):
    print('REVIEW:', review)
    print('Actual Sentiment:', sentiment)
    print('Predicted Sentiment polarity:', textblob.TextBlob(review).sentiment.polarity)
    #print ('Predicted Sentiment :', predicted_sentiments )
    print('-'*60)

REVIEW: This app sucks. My flight was canceled and decided to download the app to keep current on my connecting flight status. I ultimately missed my connection and was automatically rebooked on the next available flight twice. I only knew this because I was looking directly online. the app still had me on the first flight I missed and if I relied on the app, I would have had no clue what was going on. not to mention it is hard to navigate. I am deleting the app and going to rely on the website.
Actual Sentiment: Negative
Predicted Sentiment polarity: 0.017592592592592597
------------------------------------------------------------
REVIEW: As others have stated, app loses flight info shortly before takeoff. Crashes when attempting to view boarding pass. Frustrating...
Actual Sentiment: Negative
Predicted Sentiment polarity: -0.2333333333333333
------------------------------------------------------------
REVIEW: Had to keep adding my flight info so I could use boarding passes at each le

In [24]:
from sklearn.metrics import confusion_matrix, classification_report

In [27]:
labels = ['negative', 'positive']
print(classification_report(test_sentiments, predicted_sentiments))


              precision    recall  f1-score   support

    Negative       0.00      0.00      0.00      91.0
    Positive       0.00      0.00      0.00      11.0
    negative       0.00      0.00      0.00       0.0
    positive       0.00      0.00      0.00       0.0

    accuracy                           0.00     102.0
   macro avg       0.00      0.00      0.00     102.0
weighted avg       0.00      0.00      0.00     102.0



In [33]:
pd.DataFrame(confusion_matrix(test_sentiments, predicted_sentiments), index=labels, columns=labels)

ValueError: Shape of passed values is (4, 4), indices imply (2, 2)

# AFINN

In [34]:
from afinn import Afinn

afn = Afinn(emoticons=True)

In [35]:
for review, sentiment in zip(test_reviews[sample_review_ids], test_sentiments[sample_review_ids]):
    print('REVIEW:', review)
    print('Actual Sentiment:', sentiment)
    print('Predicted Sentiment polarity:', afn.score(review))
    print('-'*60)

REVIEW: This app sucks. My flight was canceled and decided to download the app to keep current on my connecting flight status. I ultimately missed my connection and was automatically rebooked on the next available flight twice. I only knew this because I was looking directly online. the app still had me on the first flight I missed and if I relied on the app, I would have had no clue what was going on. not to mention it is hard to navigate. I am deleting the app and going to rely on the website.
Actual Sentiment: Negative
Predicted Sentiment polarity: -9.0
------------------------------------------------------------
REVIEW: As others have stated, app loses flight info shortly before takeoff. Crashes when attempting to view boarding pass. Frustrating...
Actual Sentiment: Negative
Predicted Sentiment polarity: -2.0
------------------------------------------------------------
REVIEW: Had to keep adding my flight info so I could use boarding passes at each leg of my trip. Why does it forge

In [36]:
sentiment_polarity = [afn.score(review) for review in test_reviews]
predicted_sentiments = ['positive' if score >= 1.0 else 'negative' for score in sentiment_polarity]

In [37]:
labels = ['negative', 'positive']
print(classification_report(test_sentiments, predicted_sentiments))
pd.DataFrame(confusion_matrix(test_sentiments, predicted_sentiments), index=labels, columns=labels)

              precision    recall  f1-score   support

    Negative       0.00      0.00      0.00      91.0
    Positive       0.00      0.00      0.00      11.0
    negative       0.00      0.00      0.00       0.0
    positive       0.00      0.00      0.00       0.0

    accuracy                           0.00     102.0
   macro avg       0.00      0.00      0.00     102.0
weighted avg       0.00      0.00      0.00     102.0



ValueError: Shape of passed values is (4, 4), indices imply (2, 2)

In [39]:
stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
    doc = doc.strip()
    #doc = contractions.fix(doc)
    return doc

normalize_corpus = np.vectorize(normalize_document)

norm_corpus = normalize_corpus(test_reviews)
len(norm_corpus)

102

In [40]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

def analyze_sentiment_vader_lexicon(review, 
                                    threshold=0.1,
                                    verbose=False):    
    # analyze the sentiment for review
    analyzer = SentimentIntensityAnalyzer()
    scores = analyzer.polarity_scores(review)
    # get aggregate scores and final sentiment
    agg_score = scores['compound']
    final_sentiment = 'positive' if agg_score >= threshold\
                                   else 'negative'
    if verbose:
        # display detailed sentiment statistics
        positive = str(round(scores['pos'], 2)*100)+'%'
        final = round(agg_score, 2)
        negative = str(round(scores['neg'], 2)*100)+'%'
        neutral = str(round(scores['neu'], 2)*100)+'%'
        sentiment_frame = pd.DataFrame([[final_sentiment, final, positive,
                                        negative, neutral]],
                                        columns=pd.MultiIndex(levels=[['SENTIMENT STATS:'], 
                                                                      ['Predicted Sentiment', 'Polarity Score',
                                                                       'Positive', 'Negative', 'Neutral']], 
                                                              codes=[[0,0,0,0,0],[0,1,2,3,4]]))
        print(sentiment_frame)
    
    return final_sentiment

In [41]:
for review, sentiment in zip(norm_corpus[sample_review_ids], test_sentiments[sample_review_ids]):
    print('REVIEW:', review)
    print('Actual Sentiment:', sentiment)
    pred = analyze_sentiment_vader_lexicon(review, threshold=0.4, verbose=True)    
    print('-'*60)

REVIEW: This app sucks My flight was canceled and decided to download the app to keep current on my connecting flight status I ultimately missed my connection and was automatically rebooked on the next available flight twice I only knew this because I was looking directly online the app still had me on the first flight I missed and if I relied on the app I would have had no clue what was going on not to mention it is hard to navigate I am deleting the app and going to rely on the website
Actual Sentiment: Negative
     SENTIMENT STATS:                                         
  Predicted Sentiment Polarity Score Positive Negative Neutral
0            negative          -0.82     0.0%    11.0%   89.0%
------------------------------------------------------------
REVIEW: As others have stated app loses flight info shortly before takeoff Crashes when attempting to view boarding pass Frustrating
Actual Sentiment: Negative
     SENTIMENT STATS:                                         
  Predi

In [42]:
predicted_sentiments = [analyze_sentiment_vader_lexicon(review, threshold=0.4, verbose=False) for review in test_reviews]


In [43]:
labels = ['negative', 'positive']
print(classification_report(test_sentiments, predicted_sentiments))
pd.DataFrame(confusion_matrix(test_sentiments, predicted_sentiments), index=labels, columns=labels)

              precision    recall  f1-score   support

    Negative       0.00      0.00      0.00      91.0
    Positive       0.00      0.00      0.00      11.0
    negative       0.00      0.00      0.00       0.0
    positive       0.00      0.00      0.00       0.0

    accuracy                           0.00     102.0
   macro avg       0.00      0.00      0.00     102.0
weighted avg       0.00      0.00      0.00     102.0



ValueError: Shape of passed values is (4, 4), indices imply (2, 2)